In [1]:
import xarray as xr
from pandas.plotting import register_matplotlib_converters

register_matplotlib_converters()

%load_ext autoreload
%autoreload 2

### Paths input data

In [2]:
from ar6_ch6_rcmipfigs.constants import OUTPUT_DATA_DIR, RESULTS_DIR

# PATH_DATASET = OUTPUT_DATA_DIR + '/forcing_data_rcmip_models.nc'
# PATH_DT = OUTPUT_DATA_DIR / '/dT_data_rcmip_models.nc'
PATH_DT = OUTPUT_DATA_DIR / 'dT_data_RCMIP_recommendation.nc'

/home/sarambl/PHD/IPCC/public/AR6_CH6_RCMIPFIGSv2/ar6_ch6_rcmipfigs
/home/sarambl/PHD/IPCC/public/AR6_CH6_RCMIPFIGSv2/ar6_ch6_rcmipfigs/data_in


#### Uncertainty data from Chris

In [3]:
PATH_DT_UNCERTAINTY = OUTPUT_DATA_DIR / 'dT_uncertainty_data_FaIR_chris_ed02-3.nc'

## Set values:

In [4]:
base_period = [1995,2014]
short_term = [2021,2040]
long_term = [2041,2060]
vlong_term = [2081,2100]
periods = [short_term, long_term,vlong_term]
periods_dic  = {
    'near-term':short_term,
    'mid-term':long_term,
    'long-term':vlong_term
}

**Set reference year for temperature change:**

In [5]:
ref_year = '2020'

In [6]:
FIGURE_DIR = RESULTS_DIR / 'figures_amanda/'

TABLE_DIR = RESULTS_DIR / 'tables_amanda/'

In [7]:
from pathlib import Path
Path(FIGURE_DIR).mkdir(parents=True, exist_ok=True)
Path(TABLE_DIR).mkdir(parents=True, exist_ok=True)

In [8]:
percentile = 'percentile'
climatemodel = 'climatemodel'
scenario = 'scenario'
variable = 'variable'
time = 'time'
sum_SLCF = 'Sum SLCF (Aerosols, Methane, Ozone, HFCs)'
name_deltaT = 'Delta T'

### Define variables to look at:

In [9]:
# variables to plot:
variables_erf_comp = [
    'aerosol-total-with_bc-snow',
    'ch4',
    # 'aerosol-radiation_interactions',
    # 'aerosol-cloud_interactions',
    #'aerosol-total',
    'o3',
    'HFCs',
    #'bc_on_snow'
]
# total ERFs for anthropogenic and total:
#variables_erf_tot = ['total_anthropogenic',
#                     'total']

scenarios_fl_370 = ['ssp370', 'ssp370-lowNTCF-aerchemmip', 'ssp370-lowNTCF-gidden'  # Due to mistake here
                    ]

### Scenarios:

In [10]:
scenarios_fl = ['ssp119',
                'ssp126',
                'ssp245',
                'ssp334',
                'ssp370',
                'ssp370-lowNTCF-aerchemmip',
                #'ssp370-lowNTCF-gidden',
                'ssp370-lowNTCF-gidden',
                'ssp585']

In [11]:
median = 'median'
perc5 = '5th percentile'
perc95 = '95th percentile'
recommendation = 'recommendation'

In [12]:
ds_DT = xr.open_dataset(PATH_DT)
ds_uncertainty = xr.open_dataset(PATH_DT_UNCERTAINTY)

In [13]:
ds_DT.scenario  # .climatemodel

<xarray.DataArray 'scenario' (scenario: 11)>
array(['ssp534-over', 'ssp334', 'ssp119', 'ssp460', 'ssp585', 'ssp370',
       'ssp370-lowNTCF-aerchemmip', 'ssp126', 'ssp245', 'ssp434',
       'ssp370-lowNTCF-gidden'], dtype=object)
Coordinates:
  * scenario  (scenario) object 'ssp534-over' ... 'ssp370-lowNTCF-gidden'

In [14]:
from ar6_ch6_rcmipfigs.utils.plot import get_var_nicename

<IPython.core.display.Javascript object>

hallo


In [15]:
_str = ''
_vl = [get_var_nicename(var).split('(')[0].strip() for var in variables_erf_comp]
for var in _vl: 
    _str += f'{var}, '

# ax.set_title('Temperature change, sum SLCF  (%s)' % _str[:-2])


vn_sum = 'Sum SLCF (%s)' % _str[:-2]
print(vn_sum)
#_st = vn_sum.replace('(','').replace(')','').replace(' ','_').replace(',','')+'.csv'

_da_sum  = ds_DT[name_deltaT].sel(variable=variables_erf_comp).sum(variable)
#_da = ds_DT[name_deltaT].sel(variable=variables_erf_comp).sum(variable).sel(year=slice(int(s_y2), int(e_y2))) - ds_DT_sy
_da_sum#.assin_coord()
#_ds_check = ds_DT.copy()
ds_DT
#xr.concat([_ds_check[name_deltaT],_da_sum], dim=variable)

dd1=_da_sum.expand_dims(
    {'variable':
    [vn_sum]})
#dd1=dd1.to_dataset()

ds_DT = xr.merge([ds_DT,dd1])

Sum SLCF (Aerosols, Methane, Ozone, HFCs)


In [16]:
base_mean = ds_DT.sel(year=slice(*base_period)).mean('year')#base_period[0], base_period[1]))
bp_str = f'({base_period[0]}-{base_period[1]})'

per_means = []
per_means_anomaly = []
per_names = []
for per_n in periods_dic.keys():
    per = periods_dic[per_n]
    print(per)
    per_mean = ds_DT.sel(year=slice(*per)).mean('year')
    per_means.append(per_mean)
    per_anom = per_mean- base_mean
    anom_df = per_anom['Delta T'].squeeze().to_dataframe().drop('percentile', axis=1)
    #per_names.append()
    lab = f'{per_n} ({per[0]}-{per[1]})-({base_period[0]}-{base_period[1]})'
    per_means_anomaly.append(anom_df.rename({name_deltaT:lab}, axis=1))
    
short_term_mean = ds_DT.sel(year=slice(*short_term)).mean('year')
long_term_mean = ds_DT.sel(year=slice(*long_term)).mean('year')
base_mean#['year']

[2021, 2040]
[2041, 2060]
[2081, 2100]


/home/sarambl/miniconda3/envs/rcmip_ch6/lib/python3.7/site-packages/xarray/core/nanops.py:142: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)


<xarray.Dataset>
Dimensions:     (percentile: 1, scenario: 11, variable: 20)
Coordinates:
  * variable    (variable) object 'HFCs' ... 'volcanic'
  * scenario    (scenario) object 'ssp534-over' ... 'ssp370-lowNTCF-gidden'
  * percentile  (percentile) object 'recommendation'
Data variables:
    ERF         (scenario, variable) float64 0.009063 nan ... 0.1721 0.1444
    delta_t     float64 1.0
    Delta T     (percentile, scenario, variable) float64 0.003369 ... 0.01773

In [18]:
import pandas as pd

In [19]:
output_df = pd.concat(per_means_anomaly, axis=1)


In [20]:
fn = TABLE_DIR / 'period_anomalies.csv'
output_df.to_csv(fn)
output_df

near-term (2021-2040)-(1995-2014)  \
scenario              variable                                                                       
ssp534-over           HFCs                                                                0.032254   
                      Sum SLCF (Aerosols, Methane, Ozone, HFCs)                           0.349027   
                      aerosol-cloud_interactions                                          0.109053   
                      aerosol-radiation_interactions                                      0.086343   
                      aerosol-total                                                       0.195396   
...                                                                                            ...   
ssp370-lowNTCF-gidden solar                                                              -0.003800   
                      total                                                               0.774403   
                      total_anthropogenic                                                 0.792607   
                      total_natural                                                      -0.018204   
                      volcanic                                                           -0.014404   

                                                                 mid-term (2041-2060)-(1995-2014)  \
scenario              variable                                                                      
ssp534-over           HFCs                                                               0.070502   
                      Sum SLCF (Aerosols, Methane, Ozone, HFCs)                          0.545018   
                      aerosol-cloud_interactions                                         0.250167   
                      aerosol-radiation_interactions                                     0.091105   
                      aerosol-total                                                      0.341272   
...                                                                                           ...   
ssp370-lowNTCF-gidden solar                                                             -0.010422   
                      total                                                              1.333692   
                      total_anthropogenic                                                1.368508   
                      total_natural                                                     -0.034815   
                      volcanic                                                          -0.024393   

                                                                 long-term (2081-2100)-(1995-2014)  
scenario              variable                                                                      
ssp534-over           HFCs                                                                0.032898  
                      Sum SLCF (Aerosols, Methane, Ozone, HFCs)                           0.289846  
                      aerosol-cloud_interactions                                          0.448162  
                      aerosol-radiation_interactions                                      0.058954  
                      aerosol-total                                                       0.507116  
...                                                                                            ...  
ssp370-lowNTCF-gidden solar                                                              -0.017639  
                      total                                                               2.716104  
                      total_anthropogenic                                                 2.757280  
                      total_natural                                                      -0.041176  
                      volcanic                                                           -0.023537  

[220 rows x 3 columns]

per_anom['Delta T'].to_dataframe()#.drop('percentile', axis=1)



st_anomaly = short_term_mean-base_mean
lt_anomaly = long_term_mean- base_mean
st_anomaly_df = st_anomaly.squeeze('percentile')['Delta T'].to_dataframe().drop('percentile', axis=1)
lt_anomaly_df = lt_anomaly.squeeze('percentile')['Delta T'].to_dataframe().drop('percentile', axis=1)

bp_str = f'({base_period[0]}-{base_period[1]})'
st_str = f'({short_term[0]}-{short_term[1]})'
lt_str = f'({long_term[0]}-{long_term[1]})'

import pandas as pd

st_anomaly_df = st_anomaly_df.rename({name_deltaT: f'short term anomaly ({st_str}-{bp_str})'}, axis=1)
lt_anomaly_df = lt_anomaly_df.rename({name_deltaT: f'long term anomaly ({lt_str}-{bp_str})'}, axis=1)
output_df = pd.concat([st_anomaly_df, lt_anomaly_df], axis=1)

ds_DT.sel(year=slice(*short_term))['year']